In [68]:
from datetime import datetime, timedelta
import pandas_datareader as pdr
from finta import TA
import pandas as pd


class EquityData():
    
    """       
    The EQD class is used to get a DataFrame with equity pricing data and finacial indicators 

    Attributes
    ----------
    start_date: Optional(date object)
        Date object for the start date of your DataFrame 
        
    end_date: Optional(date object)
        Date object for the end date of your DataFrame
        
    ticker_list : Optional[list[str]]
        A list of strings providing the tickers to retrieve end of day trading data
        
    indicator_list : Optional[list[str]]
        A list of strings providing the indicators to calculate for each security in the ticker_list
        
    Examples
    --------
    e = EquityData()
    print(e.df)
    
    from datetime import datetime
    start_date = datetime.now().date() - timedelta(days=30)
    ticker_list = ['UBER']
    indicator_list = ['RSI']
    obj = EquityData(start_date=start_date, ticker_list=ticker_list, indicator_list=indicator_list)
    print(obj.df)
    
    """
    
    def __init__(self, start_date=None, end_date=None, ticker_list=None, indicator_list=None):

        if start_date == None:
            self.start_date = datetime.now().date() - timedelta(days=20)
        else:
            self.start_date = start_date            
        if end_date == None:            
            self.end_date = datetime.now().date()
        else:
            self.end_date = end_date            
        if ticker_list == None:
            self.ticker_list = ['AAPL', 'TSLA']
        else:
            self.ticker_list = ticker_list            
        if indicator_list == None:
            self.indicator_list = ['WMA', 'EMA']
        else:
            self.indicator_list = indicator_list
            
        self.df = pd.DataFrame([])
        self.get_eod_data(self)

    @staticmethod
    def get_eod_data(self):

        """
        Get EOD data for a list of tickers
        """
        eod_df = pd.DataFrame([])

        for t in self.ticker_list:
            results = pdr.DataReader(t, 'yahoo', self.start_date, self.end_date)
            results['ticker'] = t
            eod_df = eod_df.append(results)

        # Calc the list of indicators for each set of EOD data
        eod_df = eod_df.groupby(['ticker']).apply(lambda x: self.compute_analytics(x, self.indicator_list)).reset_index()

        eod_df = self.clean_data(eod_df)
        self.df = eod_df

    @staticmethod
    def compute_analytics(df, indicators):

        """
        Get calculations for each ticker in the df
        """

        for indicator in indicators:
            # Using eval so we can iterate over a list of indicators
            data = eval('TA.' + indicator + '(df)')

            # If TA returns a set -> cast set to a DataFrame
            if isinstance(data, pd.DataFrame) == False:
                data = data.to_frame()

                # To shorten the columns names I split the string 
                # and join that with the indicator name for my indicator column name
                first_string = data.columns[0].split()[0]
                data = data.rename(columns={data.columns[0]: first_string})
                data = data.add_prefix(indicator + '_')

            # Merge DataFrames based on the date
            df = df.merge(data, left_index=True, right_index=True)

        return df

    @staticmethod
    def clean_data(df):
        
        """
        Update column names to follow PEP8 and sort values
        """
        
        df = df.rename(columns={'Date': 'date', 'High': 'high', 'Low': 'low', 'Open': 'open',
                                'Close': 'close', 'Volume': 'vol', 'Adj Close': 'adj_close',
                                'SMA_41': 'sma_41', 'EMA_9': 'ema_9'})
        df = df.sort_values(by=['ticker', 'date'])
        return df


In [69]:
# obj = EquityData()
# print(obj.df)

from datetime import datetime
start_date = datetime.now().date() - timedelta(days=30)
ticker_list = ['UBER']
indicator_list = ['RSI']
obj = EquityData(start_date=start_date, ticker_list=ticker_list, indicator_list=indicator_list)
print(obj.df)

         date       high        low       open      close        vol  \
0  2020-10-23  36.990002  35.924000  36.500000  36.750000   15960600   
1  2020-10-26  36.740002  35.119999  36.500000  35.490002   12511700   
2  2020-10-27  35.700001  34.020000  35.509998  34.160000   13692100   
3  2020-10-28  33.660000  32.900002  33.619999  33.650002   15787900   
4  2020-10-29  34.935001  33.330002  33.709999  34.060001   13735300   
5  2020-10-30  34.389999  33.154999  33.910000  33.410000   13506200   
6  2020-11-02  35.049999  34.060001  34.150002  34.810001   18350500   
7  2020-11-03  36.150002  35.150002  35.270000  35.770000   16208800   
8  2020-11-04  42.150002  39.000000  40.660000  40.990002  101818200   
9  2020-11-05  42.049999  40.009998  40.330002  41.959999   34177600   
10 2020-11-06  45.380001  41.619999  42.090000  44.869999   57670700   
11 2020-11-09  49.619999  47.299999  48.009998  48.180000   61632300   
12 2020-11-10  49.360001  46.044998  46.740002  47.000000   3307